# Action Designator
This example will show the different kinds of Action Designator that are available. We will see how to create Action Designators and what they do.


Action Designator are high-level descriptions of actions which the robot should execute. 

Action Deisgnators are created from a Action Designator Descritpion, which describes the type of action as well as the parameter for this action. Parameter are given as a list of possible parameters.
For example, if you want to describe the robot moving to a table you would need a ```NavigateAction``` and a list of poses that are near the table. The Action Designator Description will then pick one of the poses and return a performable Action Designator which contains the picked pose. 



## Navigate Action
We will start with a simple example of the ```NavigateAction```. 

First, we need a BulletWorld with a robot.

In [1]:
from pycram.bullet_world import BulletWorld, Object
from pycram.pose import Pose

world = BulletWorld()
pr2 = Object("pr2", "pr2", "pr2.urdf")

Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


In [15]:
world.exit()

To move the robot we need to create a description and resolve it to an actual Designator. The description of navigation only needs a list of possible poses.

In [3]:
from pycram.designators.action_designator import NavigateAction
from pycram.pose import Pose

pose = Pose([1, 0, 0], [0, 0, 0, 1])

# This is the Designator Description
navigate_description = NavigateAction(target_locations=[pose])

# This is the performable Designator
navigate_designator = navigate_description.resolve()

What we now did was create the pose were we want to move the robot, create a description describing a navigation with a list of possible poses (in this case the list contains only one pose) and create an action designator from the description. The action designator contains the pose picked from the list of possible poses and can be performed.

In [4]:
from pycram.process_module import simulated_robot

with simulated_robot:
    navigate_designator.perform()

Every designator that is performed needs to be in an environment that specifies where to perform the designator either on the real robot or the simulated one. This environment is called ```simulated_robot``` similar there is also a ```real_robot``` environment. 

There are also decorators which do the same thing but for whole methods, they are called ```with_real_robot``` and ```with_simulated_robor```

## Move Torso
This action designator moves the torso up or down, specifically it sets the torso joint to a given value.

We start again by creating a description and resolving it to a designator. Afterwards, the designator is perfomed in a ```simulated_robot``` environment. 

In [5]:
from pycram.designators.action_designator import MoveTorsoAction
from pycram.process_module import simulated_robot

torso_pose = 0.2

torso_desig = MoveTorsoAction([torso_pose]).resolve()

with simulated_robot:
    torso_desig.perform()

## Set Gripper
As the name implies, this action designator is used to open or close the gripper. 

The procedure is similar to the last time, but this time we will shorten it a bit.

In [6]:
from pycram.designators.action_designator import SetGripperAction
from pycram.process_module import simulated_robot

gripper = "right"
motion = "open"

with simulated_robot:
    SetGripperAction(grippers=[gripper], motions=[motion]).resolve().perform()

## Park Arms
Park arms is used to move one or both arms into the default parking position.

In [7]:
from pycram.designators.action_designator import ParkArmsAction
from pycram.process_module import simulated_robot
from pycram.enums import Arms

with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()

## Pick Up and Place
Since these are depending on each other, meaning you can only place something when you picked it up beforehand, they will be shown together. 

These action designators use object designators, which will not be further explained so please check the example on object designators for more details. 

To start we need an environment in which we can pick up and place things as well as an object to pick up.

In [8]:
kitchen = Object("kitchen", "environment", "kitchen.urdf")
milk = Object("milk", "milk", "milk.stl", pose=Pose([1.3, 1, 0.9]))

world.reset_bullet_world()

Scalar element defined multiple times: limit
Scalar element defined multiple times: limit


In [9]:
from pycram.designators.action_designator import PickUpAction, PlaceAction, ParkArmsAction, MoveTorsoAction, NavigateAction
from pycram.designators.object_designator import BelieveObject
from pycram.process_module import simulated_robot
from pycram.enums import Arms
from pycram.pose import Pose

milk_desig = BelieveObject(names=["milk"])
arm ="right"

with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    
    MoveTorsoAction([0.3]).resolve().perform()
    
    NavigateAction([Pose([0.72, 0.98, 0.0], 
                     [0.0, 0.0, 0.014701099828940344, 0.9998919329926708])]).resolve().perform()
    
    PickUpAction(object_designator_description=milk_desig, 
                     arms=[arm], 
                     grasps=["right"]).resolve().perform()
    
    NavigateAction([Pose([-1.90, 0.78, 0.0], 
                     [0.0, 0.0, 0.16439898301071468, 0.9863939245479175])]).resolve().perform()
    
    PlaceAction(object_designator_description=milk_desig, 
                target_locations=[Pose([-1.20, 1.0192, 0.9624], 
                                   [0.0, 0.0, 0.6339889056055381, 0.7733421413379024])], 
                arms=[arm]).resolve().perform()

In [9]:
world.reset_bullet_world()

## Look At
Look at lets the robot look at a specific point, for example if it should look at an object for detecting. 

In [3]:
from pycram.designators.action_designator import LookAtAction
from pycram.process_module import simulated_robot
from pycram.pose import Pose

target_location = Pose([1, 0, 0.5], [0, 0, 0, 1])
with simulated_robot:
    LookAtAction(targets=[target_location]).resolve().perform()

## Detect
Detect is used to detect objects in the field of vision (FOV) of the robot. We will use the milk used in the pick up/place example, if you didn't execute that example you can spawn the milk with the following cell. The detect designator will return a resolved instance of an ObjectDesignatorDescription. 

In [5]:
milk = Object("milk", "milk", "milk.stl", pose=Pose([1.3, 1, 0.9]))

In [6]:
from pycram.designators.action_designator import DetectAction, LookAtAction, ParkArmsAction, NavigateAction
from pycram.designators.object_designator import BelieveObject
from pycram.enums import Arms
from pycram.process_module import simulated_robot
from pycram.pose import Pose

milk_desig = BelieveObject(names=["milk"])

with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    
    NavigateAction([Pose([0, 1, 0], [0, 0, 0, 1])]).resolve().perform()
    
    LookAtAction(targets=[milk_desig.resolve().pose]).resolve().perform()
    
    obj_desig = DetectAction(milk_desig).resolve().perform()
    
    print(obj_desig)

ObjectDesignatorDescription.Object(name=milk, type=milk, bullet_world_object=Object(world=<pycram.bullet_world.BulletWorld object at 0x7f18654890a0>, 
name=milk, 
type=milk, 
color=[1, 1, 1, 1], 
id=3, 
path=/home/jonas/workspace/ros/src/pycram-1/src/pycram/../../resources/cached/milk.urdf, 
joints: ..., 
links: ..., 
attachments: ..., 
cids: ..., 
original_pose=header: 
  seq: 0
  stamp: 
    secs: 1690202450
    nsecs:  95539569
  frame_id: "map"
pose: 
  position: 
    x: 1.3
    y: 1
    z: 0.9
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0, 
base_origin_shift=[ 4.15300950e-04 -6.29518181e-05  8.96554102e-02], 
local_transformer=<pycram.local_transformer.LocalTransformer object at 0x7f1860fad7c0>, 
tf_frame=milk_3, 
urdf_object: ...), _pose=<bound method Object.get_pose of Object(world=<pycram.bullet_world.BulletWorld object at 0x7f18654890a0>, 
name=milk, 
type=milk, 
color=[1, 1, 1, 1], 
id=3, 
path=/home/jonas/workspace/ros/src/pycram-1/src/pycram/../../resources/ca

## Transporting
Transporting can transport an object from its current position to another target position. It is similar to the Pick and Place plan used in the Pick-up and Place example. Since we need an Object which we can transport we spawn a milk, you don't need to do this if you alredy spawned it in a previous example.

In [7]:
kitchen = Object("kitchen", "environment", "kitchen.urdf")
milk = Object("milk", "milk", "milk.stl", pose=Pose([1.3, 1, 0.9]))

Scalar element defined multiple times: limit
Scalar element defined multiple times: limit


In [10]:
from pycram.designators.action_designator import *
from pycram.designators.object_designator import *
from pycram.process_module import simulated_robot
from pycram.pose import Pose

milk_desig = BelieveObject(names=["milk"])

with simulated_robot:
    MoveTorsoAction([0.3]).resolve().perform()
    TransportAction(milk_desig, ["left"], [Pose([-0.9, 0.9, 0.95], [0, 0, 1, 0])]).resolve().perform()

## Opening
Opening allows the robot to open a drawer, the drawer is identified by an ObjectPart designator which describes the handle of the drawer that should be grasped. 

For the moment this designator works only in the apartment environment, therefore we remove the kitchen and spawn the apartment.

In [11]:
kitchen.remove()

In [12]:
apartment = Object("apartment", "environment", "apartment.urdf")

Unknown tag "material" in /robot[@name='apartment']/link[@name='coffe_machine']/collision[1]
Unknown tag "material" in /robot[@name='apartment']/link[@name='coffe_machine']/collision[1]


In [13]:
from pycram.designators.action_designator import *
from pycram.designators.object_designator import *
from pycram.enums import Arms
from pycram.process_module import simulated_robot
from pycram.pose import Pose

apartment_desig = BelieveObject(names=["apartment"]).resolve()
handle_deisg = ObjectPart(names=["handle_cab10_t"], part_of=apartment_desig)

with simulated_robot:
    MoveTorsoAction([0.25]).resolve().perform()
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    NavigateAction([Pose([1.7474915981292725, 2.6873629093170166, 0.0],
                         [-0.0, 0.0, 0.5253598267689507, -0.850880163370435])]).resolve().perform()
    OpenAction(handle_deisg, ["right"]).resolve().perform()

## Closing
Closing lets the robot close an open drawer, like opening the drawer is identified by an ObjectPart designator describing the handle to be grasped. 

This action designator only works in the apartment environment for the moment, therefore we remove the kitchen and spawn the apartment. Additionally, we open the drawer such that we can close it with the action designator.

In [ ]:
kitchen.remove()

In [ ]:
apartment = Object("apartment", "environment", "apartment.urdf")
apartment.set_joint_state("cabinet10_drawer_top_joint", 0.4)

In [14]:
from pycram.designators.action_designator import *
from pycram.designators.object_designator import *
from pycram.enums import Arms
from pycram.process_module import simulated_robot
from pycram.pose import Pose

apartment_desig = BelieveObject(names=["apartment"]).resolve()
handle_deisg = ObjectPart(names=["handle_cab10_t"], part_of=apartment_desig)

with simulated_robot:
    MoveTorsoAction([0.25]).resolve().perform()
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    NavigateAction([Pose([1.7474915981292725, 2.6873629093170166, 0.0],
                         [-0.0, 0.0, 0.5253598267689507, -0.850880163370435])]).resolve().perform()
    CloseAction(handle_deisg, ["right"]).resolve().perform()

In [ ]:
## Pour
Pour action designator lets the robot pour a particle(not yet simulated) from one container to the other. It is an extention of pick and place example given earlier. 

This action designator only works in the apartment environment for the moment. 

In [1]:
from pycram.designators.action_designator import PickUpAction, PlaceAction, ParkArmsAction, MoveTorsoAction, \
    NavigateAction, PourAction
from pycram.designators.object_designator import *
from pycram.process_module import simulated_robot
from pycram.enums import Arms
from pycram.pose import Pose
# from pycram.designator import ObjectDesignatorDescription as odd
from pycram.bullet_world import BulletWorld, Object
from pycram.designators.location_designator import CostmapLocation, SemanticCostmapLocation
import tf


Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


In [2]:
world = BulletWorld()
world.set_gravity([0, 0, -9.8])


# source container
milk = Object("milk", "milk", "milk.stl", pose=Pose([2.4, 2.5, 1]))
# destination container
bowl = Object("bowl", "bowl", "bowl.stl", pose=Pose([2.4, 2.8, 0.98]))
pr2 = Object("pr2", "robot", "pr2.urdf", pose=Pose([1.2, 2.5, 0]))

pr2_desig = BelieveObject(names=["pr2"])
milk_desig = BelieveObject(names=["milk"])
bowl_desig = BelieveObject(names=["bowl"])

apartment = Object("apartment", "environment", "apartment.urdf")

Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown tag "material" in /robot[@name='apartment']/link[@name='coffe_machine']/collision[1]
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unkno

In [3]:
with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    MoveTorsoAction([0.3]).resolve().perform()

    # find pick up pose for source container
    pickup_pose = CostmapLocation(target=milk_desig.resolve(), reachable_for=pr2_desig.resolve()).resolve()
    pickup_arm = pickup_pose.reachable_arms[0]

    # navigate to the source container
    NavigateAction(target_locations=[pickup_pose.pose]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()

    # perform pickup action
    PickUpAction(object_designator_description=milk_desig, arms=[pickup_arm], grasps=["front"]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()
    # calculate pouring angle in quaternion
    quaternion = tf.transformations.quaternion_from_euler(90, 0, 0, axes="sxyz")
    # for simplicity calculate tilting pose where the destination container is. In this case a bowl
    tilting_pose = SemanticCostmapLocation.Location(pose=Pose( bowl.original_pose.position_as_list(), list(quaternion)))
    revert_tilting_pose = SemanticCostmapLocation.Location(pose=Pose(bowl.original_pose.position_as_list(), [0.0, 0, 0, 1]))
    # do pouring by tilting, and accept time interval
    PourAction(milk_desig, pouring_location=[tilting_pose.pose], revert_location=[revert_tilting_pose.pose],
               arms=[pickup_arm], wait_duration=5).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()
    # place the source container to its original position
    place_pose = SemanticCostmapLocation.Location(pose=Pose(milk.original_pose.position_as_list(), [0.0, 0, 0, 1.0]))
    PlaceAction(milk_desig, target_locations=[place_pose.pose], arms=[pickup_arm]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()
    # move pr2 to end pose
    end_pose = SemanticCostmapLocation.Location(pose=Pose([1, 2.5, 0], [0.0, 0, 0, 1.0]))
    NavigateAction(target_locations=[end_pose.pose]).resolve().perform()

In [4]:
world.reset_bullet_world()